Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
!git clone https://github.com/DjKesu/tortoise-tts-fast-cloning.git
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git

## **Restart the runtime!**
## Ctrl+M for Colab

In [ ]:
#@title # Setup
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

In [ ]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?" #@param {type:"string"}
#@markdown Show code for multiline text input
# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
# added very_fast preset param option, since it involves resulution with dpm++2m, expected to give best,fastest results
preset = "very_fast" #@param ["ultra_fast", "fast", "standard", "high_quality", "very_fast"]

In [ ]:
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.

#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
#@markdown Currently stored my voice clips under voices/krish/ and displaying the random rumblings of my voice.
%cd tortoise-tts-fast
%ls tortoise/voices
import IPython
IPython.display.Audio(filename='tortoise/voices/krish/1.wav')
IPython.display.Audio(filename='tortoise/voices/krish/2.wav')
IPython.display.Audio(filename='tortoise/voices/krish/3.wav')
IPython.display.Audio(filename='tortoise/voices/krish/4.wav')

In [ ]:
#@markdown Pick one of the voices from the output above
voice = 'krish' #@param {type:"string"}

#@markdown Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voices(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
#@markdown Optionally, upload use your own voice by running the next two cells. Change the name of the voice to a voice you want before running
#@markdown you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "custom"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voices(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')